## Introduction

I present a reimplementation of the following paper:

> Seppecher, P. (2012). Flexibility of wages and macroeconomic instability in an agent-based computational model with endogenous money. Macroeconomic Dynamics, 16(S2), 284-297.

After replicating the results of the original paper, I present some new questions to examine which test the robustness of the original model.

## Original model

The original paper implements the simulation using Jamel v1 software written by the author. Here I present a top-down reimplementation of the model from the original paper, with the following adjustments:

* Cash values are represented as cents and stored as `int` type, to prevent rounding and representation issues with floating-point numbers.
* Real-valued parameters are likewise represented as an arbitrary-precision `Fraction` type.
* All entities are immutable [why?]
* Entity types have no methods. Instead entity types are purely data stores, and all model mechanics are represented as separate functions. This allows me to write the code in a less fragmented manner, improving readability.

In [167]:
import enum
from enum import Enum
from fractions import Fraction
from functools import reduce
from math import floor
from random import random, sample, shuffle
from typing import NamedTuple

In [170]:
class LoanQuality(Enum):
    GOOD = enum.auto()

class Loan(NamedTuple):
    id: int
    quality: LoanQuality
    principal: int
    interest_rate: Fraction
    start_date: int
    due_date: int

class Account(NamedTuple):
    id: int
    balance: int
    loans: dict[int, Loan]

class Bank(NamedTuple):
    capital: int
    accounts: dict[int, Account]
    target_capital_ratio: Fraction = Fraction(10, 100)
    propensity_to_distribute_excess_capital: Fraction = Fraction(50, 100)
    loan_normal_interest_rate: Fraction = Fraction(5, 100)
    loan_term: int = 12

class EmploymentStatus(Enum):
    EMPLOYED = enum.auto()
    INVOLUNTARY_UNEMPLOYED = enum.auto()
    VOLUNTARY_UNEMPLOYED = enum.auto()        

class Consumption(NamedTuple):
    value: int
    quantity: int

class Household(NamedTuple):
    id: int
    account_id: int
    employment_contract_id: int
    employment_status: EmploymentStatus = EmploymentStatus.INVOLUNTARY_UNEMPLOYED
    consumption: Consumption = Consumption()
    savings_target_ratio: Fraction = Fraction(5, 100)
    propensity_to_save: Fraction = Fraction(5, 100)
    propensity_to_consume_excess: Fraction = Fraction(50, 100)
    wage_resistance: int = 12
    wage_flexibility: Fraction = Fraction(5, 100)
    job_search_size: int = 5
    purchase_search_size: int = 5

class EmploymentContract(NamedTuple):
    id: int
    household_id: int
    firm_id: int
    start_date: int
    end_date: int
    wage: int

class JobOffer(NamedTuple):
    vacancies: int
    wage: int

class Machine(NamedTuple): # Integrated
    id: int
    progress: int = 0
    value: int = 0
    production_time: int = 6
    productivity: int = 100

class GoodsForSale(NamedTuple): # GoodsOffer
    unit_price: int
    quantity: int

class Inventory(NamedTuple): # Goods
    """
    Attributes
    ----------
    value
        The total cost of manufacture of all goods.
        unit_cost = value / volume
    """
    value: int = 0
    volume: int = 0

class Firm(NamedTuple):
    id: int
    employment_contracts: dict[int, EmploymentContract]
    job_offer: JobOffer
    machinery: dict[int, Machine]
    goods_for_sale: GoodsForSale
    inventory: Inventory = Inventory()
    target_capital_ratio: Fraction = Fraction(10, 100)
    propensity_to_distribute_excess_capital: Fraction = Fraction(50, 100)
    target_inventory: int = 4_000
    propensity_to_sell: Fraction = Fraction(50, 100)
    machines: int = 10

class Simulation(NamedTuple):
    bank: Bank
    households: dict[int, Household]
    firms: dict[int, Firm]

In [1]:
def step(sim):
    sim = bank_pay_dividend(sim)
    sim = firms_pay_dividend(sim)
    sim = firms_plan_production(sim)
    sim = households_job_search(sim)
    sim = firms_production(sim)
    sim = households_consume(sim)
    sim = bank_debt_recovery(sim)
    return sim

In [122]:
def account_debt(account):
    return sum([loan.principal for loan in account.loans])

def bank_total_assets(bank):
    return sum([account_debt(account) for account in bank.accounts.values()])
    
def bank_dividend(bank):
    required_capital = bank.target_capital_ratio * bank_total_assets(bank=bank)
    excedent_capital = max(0, bank.capital - required_capital)
    return floor(excedent_capital * bank.propensity_to_distribute_excess_capital)

def bank_pay_dividend(sim):
    bank = sim.bank
    households = sim.households.values()
    if bank.capital == 0:
        raise Exception("The bank is bankrupt.")
    dividend = bank_dividend(bank=bank)
    household = sample(households, 1)[0]
    # TODO this is not how dividends are supposed to work!
    household_account = bank.accounts[household.account_id]
    return sim._replace(
        bank = bank._replace(
           capital = bank.capital - dividend,
           accounts = {
               **bank.accounts,
               **{
                   household_account.id : household_account._replace(
                       balance = household_account.balance + dividend
                   )
               }
           }
        )
    )

In [124]:
def firm_dividend(firm, firm_account):
    inventories_value = 0 # TODO
    assets = firm_account.balance + inventories_value
    capital = assets - account_debt(firm_account)
    capital_target = firm.target_capital_ratio * assets
    #debt_target = assets - capital_target # TODO what is this for?
    required_capital = firm.target_capital_ratio * assets
    excedent_capital = max(0, capital - required_capital)
    dividend = floor(excedent_capital * firm.propensity_to_distribute_excess_capital)
    return min(firm_account.balance, dividend)

def firm_pay_dividend(sim, firm):
    firm_account = sim.bank.accounts[firm.account_id]
    dividend = firm_dividend(firm=firm, firm_account=firm_account)
    household = sample(sim.households, 1)[0]
    household_account = sim.bank.accounts[household.account_id]
    return sim._replace(
        bank = bank._replace(
           accounts = {
               **bank.accounts,
               **{
                   household_account.id : household_account._replace(
                       balance = household_account.balance + dividend
                   ),
                   firm_account.id : firm_account._replace(
                       balance = firm_account.balance - dividend
                   )
               }
           }
        )
    )

def firms_pay_dividend(sim):
    reduce(firm_pay_dividend, sim.firms, sim)

In [ ]:
def bank_lend(bank, account_id, principal):
    account = bank.accounts[account_id]
    return bank._replace(
        accounts = {
            **bank.accounts,
            account_id: account._replace(
                loans = [
                    *account.loans,
                    Loan(
                        principal = principal,
                        quality = LoanQuality.GOOD,
                        interest_rate = bank.loan_normal_interest_rate,
                        term = bank.loan_term
                    )
                ],
                balance = account.balance + principal
            )
        }
    )

def firm_plan_production(sim, firm_id):
    bank = sim.bank
    firm = sim.firms[firm_id]
    
    if firm.bankrupt:
        raise Exception("Bankrupted.")
        
    # Determine production level
    inventory_ratio = Fraction(firm.inventory, firm.target_inventory * firm.max_production)
    alpha1 = random()
    alpha2 = random()
    utilization_ratio_delta = alpha1 * firm.utilization_ratio_flexibility
    target_utilization_ratio = firm.target_utilization_ratio
    if firm.inventory_ratio < 1 - alpha1 * alpha2:
        # Low level
        target_utilization_ratio += utilization_ratio_delta
    elif 1 + alpha1 * alpha2 < firm.inventory_ratio:
        # High level
        target_utilization_ratio -= utilization_ratio_delta
    # TODO? rectified
    target_utilization_ratio = min(1, target_utilization_ratio, firm.max_utilization_ratio)
    
    # Determine price
    price = firm.price
    if price == 0:
        # TODO init only
        price = 1 + 0.5 * random() # TODO * cost !
    else:
        alpha1 = random()
        alpha2 = random()
        if firm.inventory_ratio < 1 - alpha1 * alpha2:
            # Low level
            price *= 1 + alpha1 * firm.price_flexibility
        elif 1 + alpha1 * alpha2 < firm.inventory_ratio:
            # High level
            price *= 1 - alpha1 * firm.price_flexibility
        price = max(1, price)
    
    # Determine workforce
    target_employees = firm.machines * target_utilization_ratio
    target_hiring = target_employees - firm.employees
    offered_wage = firm.offered_wage
    if 0 < target_hiring:
        # TODO update offered wage
    wage_budget = target_hiring * offered_wage # TODO + current contract wages!!
    
    # Determine job offers
    # TODO need this?
    hiring = false
    if 0 < target_hiring:
        hiring = true
    
    # Determine finance
    machine_raw_materials_required = firm.technical_coefficient * firm.machine_productivity
    # TODO simplify
    raw_materials_required = max(0, 2 * firm.machines * machine_raw_materials_required - firm.raw_materials)
    raw_materials_required = min(raw_materials_required, firm.machines * machine_raw_materials_required)
    material_budget = firm.raw_materials_price * raw_materials_required
    # TODO constant price?
    production_budget = wage_budget + material_budget
    financing_need = production_budget - bank.accounts[firm.account_id].balance
    if 0 < financing_need:
        bank = bank_lend(bank=bank, account_id=firm.account_id, principal=financing_need)
    if bank.accounts[firm.account_id].balance < production_budget:
        raise Exception("Production is not financed.")
    return sim._replace(
        bank = bank,
        firms = {
            **sim.firms,
            firm_id : firm._replace(
                target_utilization_ratio = target_utilization_ratio,
                price = price,
                offered_wage = offered_wage,
                hiring = hiring
            )
        }
    )
    
def firms_plan_production(sim):
    reduce(firm_plan_production, sim.firms.keys(), sim)

In [ ]:
def household_search_employers(household, firms):
    hiring_firms = list(filter(
        lambda firm: not firm.bankrupt and firm.hiring,
        firms.values()
    ))
    found_firms = sample(hiring_firms, household.job_search_size)
    return sorted(found_firms, key=lambda firm: firm.offered_wage, reverse=true)

def household_job_search(sim, household_id):
    household = sim.households[household_id]
    
    # TODO wipe contract?
    
    # TODO if employed, update reservation wage
    reservation_wage = household.reservation_wage
    # TODO if unemployed
    unemployment_duration = household.unemployment_duration
    employment_duration = household.employment_duration
    if unemployment_duration == 0:
        # TODO init only?
        unemployment_duration = random()
    else:
        unemployment_duration += 1
        alpha1 = random()
        alpha2 = random()
        if alpha1 * household.wage_resistance < unemployment_duration:
            reservation_wage *= 1 - household.wage_flexibility * alpha2
        employment_duration = 0
    
    employment_status = household.employment_status
    # TODO if has contract, update status
    # TODO if unemployed
    employment_status = EmploymentStatus.INVOLUNTARY_UNEMPLOYED
    employers = household_search_employers(sim.firms)
    best_offer = employers[0]
    if reservation_wage <= best_offer.offered_wage:
        # TODO handle contract
    else:
        employment_status = EmploymentStatus.VOLUNTARY_UNEMPLOYED

def households_job_search(sim):
    reduce(household_job_search, sim.households.keys(), sim)

In [ ]:
def firm_production(sim, firm_id):
    firm = sim.firms[firm_id]
    
    # Pay workers
    
    # Produce
    
    # Offer goods

def firms_production(sim):
    firm_ids = list(sim.firms.keys())
    shuffle(firm_ids)
    reduce(firm_production, firm_ids, sim)

In [ ]:
def household_consume(sim, household_id):
    household = sim.households[household_id]
    
def households_consume(sim):
    household_ids = list(sim.households.keys())
    shuffle(firm_ids)
    reduce(firm_production, firm_ids, sim)

In [ ]:
def bank_debt_recovery(sim):
    

NameError: name 'self' is not defined

In [119]:
sim = Simulation(
    bank = Bank(
        capital = 1_000_00,
        accounts = {
            1: Account(
                id = 1,
                balance = 0,
                loans = [
                    Loan(
                        principal = 10
                    )
                ]
            )
        }
    ),
    households = [
        Household(
            account_id = 1
        )
    ]
)        

sim._replace(**bank_pay_dividend(**sim._asdict()))

Simulation(bank=Bank(capital=100000, accounts={1: Account(id=1, balance=49999, loans=[Loan(principal=10)])}, target_capital_ratio=Fraction(1, 10), propensity_to_distribute_excess_capital=Fraction(1, 2)), households=[Household(account_id=1)])